<a href="https://colab.research.google.com/github/Madinsh/project/blob/main/Kemengerus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Установка зависимостей для Colab (выполнить один раз)
!pip install selenium webdriver-manager
!apt-get update
!apt install chromium-chromedriver

# ---- ОСНОВНОЙ КОД ----
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# Настройка для Google Colab
options = Options()
options.add_argument("--headless") # Обязательно: без интерфейса
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Путь к установленному хрому в Colab
driver = webdriver.Chrome(options=options)

print("Браузер успешно запущен в Colab!")
# Дальше ваш код парсинга...
driver.get("https://kolesa.kz/cars/almaty/")
print(driver.title)
driver.quit()
# Настройки
BASE_URL = "https://kolesa.kz/cars/almaty/"
PAGES_TO_SCRAPE = 2  # Сколько страниц пройти

# Настройка браузера
options = webdriver.ChromeOptions()
# options.add_argument("--headless") # Раскомментируйте, чтобы браузер не открывался визуально (но для отладки лучше видеть)
options.add_argument("--disable-blink-features=AutomationControlled") # Скрываем, что это автотест
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

print("Запуск браузера...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_ads = []

try:
    for page in range(1, PAGES_TO_SCRAPE + 1):
        url = f"{BASE_URL}?page={page}"
        print(f"\n---> Открываем страницу: {page} ({url})")

        driver.get(url)
        time.sleep(5)  # Даем время на загрузку сайта и выполнение скриптов защиты

        # Находим блоки с объявлениями.
        # На Kolesa.kz карточки обычно имеют класс 'a-card' или содержатся в 'a-list'
        # Используем более универсальный поиск по классу, содержащему 'a-card'
        ads = driver.find_elements(By.CLASS_NAME, "a-card__info")

        if not ads:
            print("⚠️ Объявления не найдены! Возможно, стоит капча или изменилась верстка.")
            # Сохраним скриншот для проверки, если что-то пойдет не так
            driver.save_screenshot(f"debug_page_{page}.png")
            continue

        print(f"Найдено объявлений на странице: {len(ads)}")

        for ad in ads:
            try:
                # Извлекаем название
                try:
                    title = ad.find_element(By.CLASS_NAME, "a-card__title").text
                except:
                    title = "Нет названия"

                # Извлекаем цену
                try:
                    price_text = ad.find_element(By.CLASS_NAME, "a-card__price").text
                    price = "".join(filter(str.isdigit, price_text)) # Оставляем только цифры
                except:
                    price = "0"

                # Извлекаем регион/дату (блок subtitle)
                try:
                    region = ad.find_element(By.CLASS_NAME, "a-card__subtitle").text
                except:
                    region = "-"

                all_ads.append({
                    "Title": title,
                    "Price": price,
                    "Region": region,
                    "Link": url  # Ссылка на страницу поиска, можно доработать для конкретного авто
                })
            except Exception as e:
                print(f"Ошибка парсинга отдельного объявления: {e}")

except Exception as e:
    print(f"Критическая ошибка: {e}")

finally:
    driver.quit() # Закрываем браузер
    print("Браузер закрыт.")

# Сохранение
if all_ads:
    df = pd.DataFrame(all_ads)
    df.to_csv("kolesa_selenium.csv", index=False, encoding="utf-8-sig")
    print(f"\n[ГОТОВО] Сохранено {len(df)} строк в 'kolesa_selenium.csv'")
    print(df.head())
else:
    print("\n[ПУСТО] Не удалось собрать данные.")

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,902 kB]
Fetched 5,628 kB in 2s (2,653 kB/s)
Reading packag

SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x559d3e5e316a <unknown>
#1 0x559d3e060c4b <unknown>
#2 0x559d3e09ad1d <unknown>
#3 0x559d3e0966d3 <unknown>
#4 0x559d3e0e624c <unknown>
#5 0x559d3e0e596c <unknown>
#6 0x559d3e0a4c42 <unknown>
#7 0x559d3e0a58f1 <unknown>
#8 0x559d3e5abf09 <unknown>
#9 0x559d3e5aee4d <unknown>
#10 0x559d3e594c51 <unknown>
#11 0x559d3e5afa2b <unknown>
#12 0x559d3e57ba20 <unknown>
#13 0x559d3e5d0a78 <unknown>
#14 0x559d3e5d0c49 <unknown>
#15 0x559d3e5e24c3 <unknown>
#16 0x7c72032a4ac3 <unknown>


In [ ]:
# ============================================
# PART 2: CLEANING + STYLED TABLE (and EDA)
# ============================================

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# ---------- reading raw data ----------
raw_file = "ny_cities_raw.csv"
df = pd.read_csv(raw_file)

print("\n[READ RAW DATA]")
print("Columns:", list(df.columns))
print(df.head().to_string(index=False))

# ---------- helper for numbers ----------
def clean_num(value: str):
    if not isinstance(value, str):
        value = str(value)
    value = value.replace(",", "")
    value = value.replace("%", "")
    return pd.to_numeric(value, errors="coerce")

# ---------- Fix broken header ----------
rename_map = {
    "2025 Pop.â\x80\x94": "2025 Pop.",
    "2025 Pop.â\x80\x93": "2025 Pop.",
    "2025 Pop.â": "2025 Pop.",
    "2025 Pop.ã€–": "2025 Pop.",
    "2025 Pop.â": "2025 Pop.",
    "2025 Pop.": "2025 Pop.",

    "2024 Pop.â\x80\x94": "2024 Pop.",
    "2024 Pop.â\x80\x93": "2024 Pop.",
    "2024 Pop.â": "2024 Pop.",
    "2024 Pop.â": "2024 Pop.",
    "2024 Pop.": "2024 Pop.",

    "Area (miÂ²)": "Area (mi²)",
    "Area (miÃ‚Â²)": "Area (mi²)",
    "Area (miÂ)": "Area (mi²)",

    "Annual Changeâ\x80\x93": "Annual Change",
    "Annual Changeâ": "Annual Change",
    "Annual Changeâ": "Annual Change",
}
df.rename(columns=rename_map, inplace=True)

df.columns = df.columns.str.encode('latin1', 'ignore').str.decode('utf-8', 'ignore')
df.columns = df.columns.str.replace(r"[^\x00-\x7F]+", "", regex=True)

# ---------- unification of column names ----------
col_unify = {}
for c in df.columns:
    lc = c.lower()
    if "city" == lc:
        col_unify[c] = "City"
    elif "state" == lc:
        col_unify[c] = "State"
    elif "rank" in lc:
        col_unify[c] = "Rank"
    elif "2025" in lc and "pop" in lc:
        col_unify[c] = "Pop2025"
    elif "2024" in lc and "pop" in lc:
        col_unify[c] = "Pop2024"
    elif "annual" in lc and "change" in lc:
        col_unify[c] = "AnnualChange"
    elif "density" in lc:
        col_unify[c] = "Density"
    elif "area" in lc:
        col_unify[c] = "AreaMi2"
    elif "type" in lc:
        col_unify[c] = "Type"

df.rename(columns=col_unify, inplace=True)

# ---------- nice state name ----------
df["State"] = (
    df["State"]
    .str.replace("-", " ")
    .str.title()
)

# ---------- converting numeric columns ----------
numeric_like_columns = []
for col in df.columns:
    sample_text = "".join(df[col].astype(str).head(10).tolist())
    if any(ch.isdigit() for ch in sample_text):
        numeric_like_columns.append(col)

for col in numeric_like_columns:
    df[col] = df[col].astype(str).apply(clean_num)

print("\n[CLEANED DATA INFO]")
print(df.info())
print(df.head().to_string(index=False))

# ---------- basic stats ----------
print("\n=== Descriptive statistics (describe) ===")
print(df.describe(include="all"))
print("\n=== Missing values per column ===")
print(df.isna().sum())
print("\nNumber of duplicated rows:", df.duplicated().sum())

# можно сохранить уже очищенный вариант отдельно
clean_file = "ny_cities_clean.csv"
df.to_csv(clean_file, index=False, encoding="utf-8-sig")
print(f"\nCleaned dataset saved to: {clean_file}")

# ============================================
# STYLED TABLE
# ============================================

pretty_columns_order = [
    "City",
    "State",
    "Rank",
    "Pop2025",
    "Pop2024",
    "AnnualChange",
    "Density",
    "AreaMi2",
    "Type"
]
pretty_columns_order = [c for c in pretty_columns_order if c in df.columns]
df_display = df[pretty_columns_order].copy()

def highlight_change(val):
    try:
        v = float(val)
    except:
        return ""
    if v > 0:
        return "color: green; font-weight: bold;"
    elif v < 0:
        return "color: red; font-weight: bold;"
    else:
        return ""

pop_cols  = [c for c in df_display.columns if "Pop20" in c]
dens_cols = ["Density"] if "Density" in df_display.columns else []

styled_C = (
    df_display.style
    .set_table_styles(
        [
            {"selector": "th",
             "props": [
                 ("background-color", "#333333"),
                 ("color", "white"),
                 ("text-align", "center"),
                 ("font-weight", "bold"),
                 ("padding", "6px")
             ]},
            {"selector": "th.row_heading, td.row_heading",
             "props": [
                 ("background-color", "#333333"),
                 ("color", "white"),
                 ("font-weight", "bold"),
                 ("border-right", "1px solid #ffffff")
             ]}
        ]
    )
    .set_properties(**{
        "font-family": "Calibri, Arial, sans-serif",
        "font-size": "18px",
        "border": "1px solid #dddddd",
        "padding": "3px 5px"
    })
    .background_gradient(cmap="Blues", subset=pop_cols)
    .background_gradient(cmap="Greens", subset=dens_cols)
    .applymap(highlight_change, subset=["AnnualChange"])
    .format({
        "Rank": "{:,.0f}".format,
        "Pop2025": "{:,.0f}".format,
        "Pop2024": "{:,.0f}".format,
        "Density": "{:,.0f}".format,
        "AreaMi2": "{:,.1f}".format,
        "AnnualChange": "{:+.2f}%".format
    })
)

title_html = """
<h2 style="font-family: Calibri, Arial, sans-serif; margin-bottom: 4px; color:#333;">
    New York Cities by Population 2025 (Scraped from WorldPopulationReview)
</h2>
<p style="font-family: Calibri, Arial, sans-serif; margin-top: 0; font-size: 12px; color: #555;">
    Dataset built using Python, requests, BeautifulSoup, and pandas (state of New York).
</p>
"""

print("\n=== STYLED TABLE PREVIEW ===")
display(HTML(title_html))
display(styled_C)


In [ ]:
'''
res1 = requests.get(url)
res2 = requests.get(url, headers=headers)

print(len(res1.text), len(res2.text))
'''

In [ ]:
# ============================================
# PART 3 — Exploratory Data Analysis (EDA)
# ============================================

import matplotlib.pyplot as plt
import seaborn as sns

eda = df.copy()

# ------------------------------------------------
# 1) DESCRIPTIVE STATISTICS
# ------------------------------------------------

print("\n======= DESCRIPTIVE STATISTICS =======")
print(eda.describe(include='all'))

print("\nMissing values:")
print(eda.isna().sum())

print("\nCorrelation matrix (numeric features):")
print(eda.corr(numeric_only=True))


# ------------------------------------------------
# 2) VISUALIZATIONS (6+ meaningful plots)
# ------------------------------------------------



eda_df = df.copy()

# 1) Top 15 fastest-growing cities (by AnnualChange)
growing = eda_df.sort_values("AnnualChange", ascending=False).head(15)

plt.figure(figsize=(10,5))
plt.barh(growing["City"], growing["AnnualChange"], color="green")
plt.xlabel("Annual Change (%)")
plt.title("Top 15 Fastest-Growing US Cities")
plt.gca().invert_yaxis()  # biggest on top
plt.tight_layout()
plt.show()

# 2) Top 15 fastest-shrinking cities (negative AnnualChange)
shrinking = eda_df.sort_values("AnnualChange").head(15)

plt.figure(figsize=(10,5))
plt.barh(shrinking["City"], shrinking["AnnualChange"], color="red")
plt.xlabel("Annual Change (%)")
plt.title("Top 15 Fastest-Shrinking US Cities")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 3) Top 10 largest cities by 2025 population (bar chart)
top10 = eda_df.sort_values("Pop2025", ascending=False).head(10)

plt.figure()
plt.bar(top10["City"], top10["Pop2025"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Population 2025")
plt.title("Top 10 Largest US Cities by Population (2025)")
plt.tight_layout()
plt.show()

# 4) Scatter: Density vs Population
plt.figure()
plt.scatter(eda_df["Pop2025"], eda_df["Density"], s=5)
plt.xlabel("Population 2025")
plt.ylabel("Population Density (people per mi²)")
plt.title("City Population vs. Density - All States")
plt.tight_layout()
plt.show()

# 5) BAR + LINE PLOT — Select top 15 cities by 2025 population
temp = df.copy()
temp["Delta"] = temp["Pop2025"] - temp["Pop2024"]
top_delta = temp.sort_values("Delta", ascending=False).head(15)

plt.figure(figsize=(10,6))
plt.plot(top_delta["City"], top_delta["Pop2024"], marker="o", label="2024")
plt.plot(top_delta["City"], top_delta["Pop2025"], marker="o", label="2025")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Population")
plt.title("Population Change 2024 → 2025 (Top 15 Cities by Increase)")
plt.legend()
plt.tight_layout()
plt.show()


# 6) PIE CHART — Share of Top 10 Cities by Population (2025)
top10 = df.sort_values("Pop2025", ascending=False).head(10)

plt.figure(figsize=(8, 8))
plt.pie(
    top10["Pop2025"],
    labels=top10["City"],
    autopct="%1.1f%%",
    startangle=140,
    textprops={'fontsize': 10}
)
plt.title("Share of Top 10 Most Populated Cities in New York (2025)")
plt.tight_layout()
plt.show()




In [ ]:
# ------------------------------------------------
# 3) ANALYSIS QUESTIONS
# ------------------------------------------------

print("\n======= ANALYSIS QUESTIONS =======")

# Q1) Cities with highest/highest population?
print("\nQ1: Largest and smallest cities by 2025 population:")
print("Largest:")
print(eda.sort_values("Pop2025", ascending=False).head(5)[["City","State","Pop2025"]])
print("Smallest:")
print(eda.sort_values("Pop2025").head(5)[["City","State","Pop2025"]])

# Q2) Which state dominates by number of cities?
print("\nQ2: State with the most cities:")
print(eda["State"].value_counts().head(5))

# Q3) Which state has the highest average city population?
print("\nQ3: States with highest average city population:")
print(eda.groupby("State")["Pop2025"].mean().sort_values(ascending=False).head(5))

# Q4) Fastest-growing cities
print("\nQ4: Top 5 fastest-growing cities:")
print(eda.sort_values("AnnualChange", ascending=False).head(5)[["City","State","AnnualChange"]])

# Q5) Fastest-shrinking cities
print("\nQ5: Top 5 shrinking cities:")
print(eda.sort_values("AnnualChange").head(5)[["City","State","AnnualChange"]])

# Q6) Do bigger cities grow faster? (Pop2025 vs AnnualChange correlation)
print("\nQ6: Correlation between population and growth rate:")
print(eda["Pop2025"].corr(eda["AnnualChange"]))

# Q7) Relationship between population and density
print("\nQ7: Correlation between population and density:")
print(eda["Pop2025"].corr(eda["Density"]))

# Q8) Which states have the densest cities?
print("\nQ8: States with highest average density:")
print(eda.groupby("State")["Density"].mean().sort_values(ascending=False).head(5))

In [ ]:
'''
import requests

url = "https://www.imdb.com/chart/top/"

headers = {
    "User-Agent": "Mozilla/5.0"
}
res = requests.get(url, headers=headers)
print(res.text[:1000])
print('''



''')
res = requests.get(url)
print(res.text[:1000])
'''

In [ ]:
'''requests.get("https://amazon.com").text
print('''



''')
requests.get("https://amazon.com", headers={"User-Agent": "Mozilla/5.0"}).text'''
